<a href="https://colab.research.google.com/github/sergak0/AIIJC/blob/main/Lisa/nb_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Зависимости



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pymorphy2[fast]

In [3]:
import random
import re
import string
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
import warnings 
from sklearn.linear_model import LogisticRegression
from PIL import Image
from IPython.display import clear_output
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from typing import List
from sklearn.metrics import classification_report
import pickle
from typing import List
import nltk
import string
import pymorphy2
import codecs

In [4]:
warnings.filterwarnings(action='ignore',category=UserWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='gensim')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='smart_open') 
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='sklearn')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='scipy')    

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
class Preparator():
  def __init__(self):
      self.morph = pymorphy2.MorphAnalyzer()
      self.tokenizer = nltk.WordPunctTokenizer()
      self.stopwords = set(line.strip() for line in codecs.open('/content/drive/MyDrive/aiijc_sber/all_stopwords.txt', "r", "utf_8_sig").readlines())
      self.bigram_mod = gensim.models.Phrases.load("/content/drive/MyDrive/aiijc_sber/bigram_model.pkl")

  def prepare_corp(self, news_list: List[str]):
      return [self.newstext2token(news_text) for news_text in news_list]

  def newstext2token(self, news_text: str):
      tokens = self.tokenizer.tokenize(news_text.lower())
      tokens_with_no_punct = [self.morph.parse(w)[0].normal_form for w in tokens if all(c in 'йцукенгшщзхъёфывапролджэячсмитьбю' for c in w)]
      tokens_base_forms = [w for w in tokens_with_no_punct if w not in self.stopwords]
      tokens_last = [w for w in tokens_base_forms if len(w)>1]
      tokens_bigrammed = self.make_bigrams(tokens_last)
      return ' '.join(tokens_bigrammed)

  def make_bigrams(self, doc):
      return self.bigram_mod[doc]

In [7]:
class BayesModel():
  
  def __init__(self,
               pretrained = True,
               vect_path: str = '/content/drive/MyDrive/aiijc_sber/nb/nb_vect.sav',
               tfidf_path: str = '/content/drive/MyDrive/aiijc_sber/nb/nb_tfidf.sav',
               clf_path: str = '/content/drive/MyDrive/aiijc_sber/nb/nb_clf.sav'):

    if not pretrained:
        self.vect = CountVectorizer()
        self.tfidf = TfidfTransformer()
        self.clf = MultinomialNB()
    else:
        vect = pickle.load(open(vect_path, 'rb'))
        tfidf = pickle.load(open(tfidf_path, 'rb'))
        clf = pickle.load(open(clf_path, 'rb'))

    self.nb = Pipeline([('vect', vect),
                  ('tfidf', tfidf),
                  ('clf', clf),
                  ])


  def prepare_dataset(self, data: pd.DataFrame):
    self.data = data
    self.data['x'] = Preparator().prepare_corp(self.data['texts'])
    return self.data

  def train(self, X_train, Y_train):
    self.nb.fit(X_train, Y_train)
  
  def predict(self, X_test: List[str]):
    return self.nb.predict(X_test)
  
  def show_metrics(self, X_test, Y_test):
    y_pred = self.predict(X_test)
    print('accuracy %s' % accuracy_score(y_pred, Y_test))
    print(classification_report(Y_test, y_pred, target_names=['животные', "музыка", "спорт", "литература"]))

  def score(self, X_test, Y_test):
    return self.nb.score(X_test, Y_test)


  def save_models(self,
                  vect_path: str = '/content/drive/MyDrive/aiijc_sber/nb/nb_vect.sav',
                  tfidf_path: str = '/content/drive/MyDrive/aiijc_sber/nb/nb_tfidf.sav',
                  clf_path: str = '/content/drive/MyDrive/aiijc_sber/nb/nb_clf.sav'):
    pickle.dump(self.vect, open(vect_path, 'wb'))
    pickle.dump(self.tfidf, open(tfidf_path, 'wb'))
    pickle.dump(self.clf, open(clf_path, 'wb'))

    

In [8]:
nb_model = BayesModel()

In [9]:
tests = pd.read_csv('/content/drive/MyDrive/aiijc_sber/test.csv')
tests.columns = ['id', 'texts']
tests.head()

,id,texts
0,0,Вопрос 1 Денис готовится провести соревнования...
1,1,Реши задачу.\nСколько коробок корма для собак ...
2,2,"Если групп элементов несколько, то применяется..."
3,3,Вопрос1 ...
4,4,Для наглядной геометрической иллюстрации объём...


In [10]:
tests['prediction'] = nb_model.predict(nb_model.prepare_dataset(tests).x)

In [11]:
def decode(label):
  return ['животные', "музыка", "спорт", "литература"][label]

In [12]:
for id, st in tests[:10].iterrows():
  print(decode(st['prediction']))
  print(st.texts)
  print()

спорт
Вопрос 1 Денис готовится провести соревнования по настольному хоккею и решил распечатать недостающие фигурки на принторе . Для изготовления одной 3D-модели у Дениса слишком мало времени, поэтому он решает использовать 2 принтера сразу, а потом склеить полученные детали. У специалиста сейчас в наличии 8 принтеров. Сколько у него есть вариантов выбора, если сначала он настраивает один из принтеров, а за ним — второй? 

музыка
Реши задачу.
Сколько коробок корма для собак поместится в ящике, если все стороны ящика соответственно в 12 раз больше сторон коробки корма?

спорт
Если групп элементов несколько, то применяется закон сложения. Если необходимо выбрать  элементы из одной группы и элементы из другой, то количество способов выбрать элементы из обеих групп будет равно 
Исходя из этого, реши задачу.
Для получения зачета по физической культуре учащимся нужно выбрать направление в котром бы они занимались в течение года и сдавали зачет. По легкой отелитики предполагается 1 группа,  п